### Data

In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [2]:
from diffusion.data_loaders.backflip_dataset import BackflipMotionDataset
dataset = BackflipMotionDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_backflip.txt")
len(dataset), dataset[0].shape

(2900, torch.Size([29, 70]))

### Model NN

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # not used in the final model
        x = x + self.pe[:x.shape[0], :]
        return self.dropout(x)


class TimestepEmbedder(nn.Module):
    def __init__(self, latent_dim, sequence_pos_encoder):
        super().__init__()
        self.latent_dim = latent_dim
        self.sequence_pos_encoder = sequence_pos_encoder

        time_embed_dim = self.latent_dim
        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

    def forward(self, timesteps):
        return self.time_embed(self.sequence_pos_encoder.pe[timesteps])
    


### Model 1

In [4]:
class MotionTransformer(nn.Module):
    def __init__(self, nfeats, latent_dim=256, ff_size=1024, num_layers=8, num_heads=4, dropout=0.1, activation="gelu"):
        super(MotionTransformer, self).__init__()
        
        self.nfeats = nfeats
        self.latent_dim = latent_dim
        self.ff_size = ff_size  
        self.dropout = dropout

        self.inputEmbedding = nn.Linear(self.nfeats, self.latent_dim)
        self.sequence_pos_encoder = PositionalEncoding(self.latent_dim, self.dropout)
        self.embed_timestep = TimestepEmbedder(self.latent_dim, self.sequence_pos_encoder)

        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model=self.latent_dim, nhead=num_heads, 
                                                    dim_feedforward=ff_size, dropout=dropout, activation=activation, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Output Linear Layer
        self.outputEmbedding = nn.Linear(self.latent_dim, self.nfeats)

    def forward(self, x: torch.Tensor, timesteps, y=None):
        """
        x: [batch_size, max_frames, n_feats], denoted x_t in the paper
        timesteps: [batch_size] (int)
        """
        # x: [batch_size, seq_len, nfeats]
        emb = self.embed_timestep(timesteps)  # [bs, seq_len, time_embed_dim]
        # print("Emb", emb.shape)

        # Input process
        x = x.float()
        # print(x.dtype, x.shape)
        x = self.inputEmbedding(x)  # [bs, seq_len, d]
        # print("Input Embedding", x.shape)

        # Transformer Encoder
        # adding the timestep embed
        xseq = torch.cat((emb, x), axis=1)  # [bs, n_frames+1, d]
        # print("Concat x and zkx", xseq.shape)

        xseq = self.sequence_pos_encoder(xseq)  # [bs, n_frames+1, d]
        # print("Sequence Pos Encoder", xseq.shape)
        
        output = self.transformer_encoder(xseq)[:, 1:, :]  # , src_key_padding_mask=~maskseq)  # [bs, n_frames, d]
        # print("Transformer Encoder", output.shape)

        # Output Linear
        output = self.outputEmbedding(output)  # [bs, n_frames, n_feats]
        # print("Output Embedding", output.shape)

        return output

In [5]:
len(dataset), dataset[0].shape

(2900, torch.Size([29, 70]))

In [6]:
import torch.optim as optim
from torch.utils.data import DataLoader

batch_size = 1
dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, drop_last=True)

nfeats = dataset[0].shape[1]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MotionTransformer(nfeats=nfeats, latent_dim=64, ff_size=256, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)

In [7]:
for it, batch in enumerate(dataloader):
    batch = batch.to(device)
    print(batch.shape)


torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 29, 70])
torch.Size([1, 2

In [8]:
from diffusion.diffusion import gaussian_diffusion as gd
from diffusion.diffusion.respace import SpacedDiffusion, space_timesteps

def create_gaussian_diffusion(
        diffusion_steps, # number eg 1000
        noise_schedule, # can be 'linear', 'cosine'
        sigma_small, # default True
        lambda_vel, lambda_rcxyz, lambda_fc # for geometric loss, we don't have fc, default 1 for rest
        ):
    # default params
    predict_xstart = True  # we always predict x_start (a.k.a. x0), that's our deal!
    steps = diffusion_steps
    scale_beta = 1.  # no scaling
    timestep_respacing = ''  # can be used for ddim sampling, we don't use it.
    learn_sigma = False
    rescale_timesteps = False

    betas = gd.get_named_beta_schedule(noise_schedule, steps, scale_beta)
    loss_type = gd.LossType.MSE

    if not timestep_respacing:
        timestep_respacing = [steps]

    return SpacedDiffusion(
        use_timesteps=space_timesteps(steps, timestep_respacing),
        betas=betas,
        model_mean_type=(
            gd.ModelMeanType.EPSILON if not predict_xstart else gd.ModelMeanType.START_X
        ),
        model_var_type=(
            (
                gd.ModelVarType.FIXED_LARGE
                if not sigma_small
                else gd.ModelVarType.FIXED_SMALL
            )
            if not learn_sigma
            else gd.ModelVarType.LEARNED_RANGE
        ),
        loss_type=loss_type,
        rescale_timesteps=rescale_timesteps,
        lambda_vel=lambda_vel,
        lambda_rcxyz=lambda_rcxyz,
        lambda_fc=lambda_fc,
    )

In [9]:
model = MotionTransformer(nfeats=nfeats, latent_dim=64, ff_size=256, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)
diffusion = create_gaussian_diffusion(diffusion_steps=1000, noise_schedule="cosine", sigma_small=True, lambda_vel=1, lambda_rcxyz=1, lambda_fc=1)

In [10]:
class DefaultArgs:
    def __init__(self, save_dir, model_path, eval_model_path):
        # Base options
        self.cuda = True
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.seed = 10
        self.batch_size = 1

        # Diffusion options
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.sigma_small = True

        # Model options
        self.arch = 'trans_enc'
        self.emb_trans_dec = False
        self.layers = 8
        self.latent_dim = 512
        self.cond_mask_prob = 0.1
        self.lambda_rcxyz = 0.0
        self.lambda_vel = 0.0
        self.lambda_fc = 0.0
        self.unconstrained = False  # This is inferred from the 'action' parameter

        # Data options
        self.dataset = 'humanml'
        self.data_dir = ""

        # Training options
        self.save_dir = save_dir
        self.overwrite = False
        self.train_platform_type = 'NoPlatform'
        self.lr = 1e-4
        self.weight_decay = 0.0
        self.lr_anneal_steps = 0
        self.eval_batch_size = 16
        self.eval_split = 'test'
        self.eval_during_training = False
        self.eval_rep_times = 3
        self.eval_num_samples = 1000
        self.log_interval = 1000
        self.save_interval = 5000
        self.num_steps = 30000
        # self.num_frames = 29
        self.resume_checkpoint = ""

        # Sampling options
        self.model_path = model_path
        self.output_dir = ''
        self.num_samples = 10
        self.num_repetitions = 3
        self.guidance_param = 2.5

        # Generate options
        self.motion_length = 6.0
        self.input_text = ''
        self.action_file = ''
        self.text_prompt = ''
        self.action_name = ''

        # Edit options
        self.edit_mode = 'in_between'
        self.text_condition = ''
        self.prefix_end = 0.25
        self.suffix_start = 0.75

        # Evaluation options
        self.eval_model_path = eval_model_path
        self.eval_mode = 'wo_mm'
        self.eval_guidance_param = 2.5


In [11]:
args = DefaultArgs(save_dir="/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/more_mixed_data", model_path="/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/more_mixed_data/model.pt", eval_model_path="/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/more_mixed_data/model.pt")
args.device

'cuda:0'

In [12]:
len(dataloader)

2900

In [13]:
from train.training_loop import TrainLoop
TrainLoop(args, None , model, diffusion, dataloader).run_loop()


Starting epoch 0/11


  0%|          | 0/2900 [00:00<?, ?it/s]

Logging to /tmp/openai-2023-12-08-09-10-09-983506
-------------------------
| grad_norm  | 5.05e+03 |
| loss       | 1.21e+04 |
| loss_q0    | 1.21e+04 |
| param_norm | 54       |
| samples    | 1        |
| step       | 0        |
-------------------------
step[0]: loss[12083.28437]
saving model...


  0%|          | 1/2900 [00:00<35:47,  1.35it/s]

Skipping evaluation for now.


...

-------------------------
| grad_norm  | 2.34e+03 |
| loss       | 847      |
| loss_q0    | 35.4     |
| loss_q1    | 37.8     |
| loss_q2    | 65.8     |
| loss_q3    | 3.13e+03 |
| param_norm | 58       |
| samples    | 3.1e+04  |
| step       | 3.1e+04  |
-------------------------


 69%|██████▉   | 2003/2900 [01:41<00:44, 20.03it/s]

step[31000]: loss[846.60812]


100%|██████████| 2900/2900 [02:26<00:00, 19.83it/s]

saving model...
Skipping evaluation for now.


### Sampling

In [14]:
num_frames = dataset[0].shape[0]
num_feats = dataset[0].shape[1]

In [15]:
args.num_repetitions

3

In [16]:
sample_batch_size = 1

all_motions = []
for rep_i in range(args.num_repetitions):
    print(f'### Sampling [repetitions #{rep_i}]')

    sample_fn = diffusion.p_sample_loop

    sample = sample_fn(
        model,
        (sample_batch_size, num_frames, num_feats),
        clip_denoised=False,
        model_kwargs={"y": {}},
        skip_timesteps=0,  # 0 is the default value - i.e. don't skip any step
        init_image=None,
        progress=True,
        dump_steps=None,
        noise=None,
        const_noise=False,
    )

    all_motions.append(sample.cpu().numpy())

/home/kenji/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Sampling [repetitions #0]


100%|██████████| 1000/1000 [00:04<00:00, 202.84it/s]


### Sampling [repetitions #1]


100%|██████████| 1000/1000 [00:05<00:00, 192.34it/s]


### Sampling [repetitions #2]


100%|██████████| 1000/1000 [00:05<00:00, 193.57it/s]


In [17]:
all_motions = np.concatenate(all_motions, axis=0)
all_motions.shape

(3, 29, 69)

### Port motion out to MocapV2 play

In [19]:
import numpy as np

def save_motions(all_motions, output_dir):
    for i, motion in enumerate(all_motions):
        filename = f"motion_{i}.npy"
        filepath = os.path.join(output_dir, filename)
        np.save(filepath, motion)
        print(f"Motion {i} saved as {filename}")

save_motions(all_motions, "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/more_mixed_data/sampled_motions/")

Motion 0 saved as motion_0.npy
Motion 1 saved as motion_1.npy
Motion 2 saved as motion_2.npy


### Progress

- Not using enough dimensions
- Data not in the right format
- Model not in the right format
- Loss is just MSE, we can aim to produce the same first pos concat with velocities
    - Currently we technically use pure geometric loss
    - Can make the model use some logits first that we can use to manually convert to configs and velocities, then we have another loss criteria
- Uneven joint config and velocity